# Send alert emails about packet loss based on alarms and user subscribing

This notebook is run by a cron job every hour, and its purpose is to send alert emails about packet loss for user specified site(s) based on alarms and user subscribing records. 

This notebook works following such procedure: 

(1) Get all the alarms of type packetloss for the past hour (call it NEW) and past past hour (call it OLD) from Elasticsearch

(2) Get the user subscribing records from Google Sheets calling APIs in subscribers.py

(3) Process the alarms data and subscribing data to make them easier to use for this monitoring task

(4) TN_old means total number of alarmed links involving a specific site ip (no matter from it or to it) for OLD time period

(5) TN_new means total number of alarmed links involving a specific site ip (no matter from it or to it) for NEW time period

(6) TN_delta means the change of value from TN_old to TN_new. We need to compare TN_delta v.s. +N and v.s. -N (tune N later)

(7) If a site ip never occurs in NEW and OLD, then it must be totally fine, and we do not care about it at all (TN_old == TN_new == TN_delta == 0)

(8) If a site ip occurs in NEW or OLD or both, then we may have TN_delta > 0 or == 0 or < 0 for this site ip, so we want to take a closer look at this site ip, so we do (9) (10) (11)

(9) If TN_delta >= +N, then overall the links connected to this site are becoming worse, so we send email

(10) If TN_delta <= -N, then overall the links connected to this site are becoming better, so we send email

(11) Otherwise, the overall status for this site is not changing or just changing slightly, so we do not send email

(12) In order to send email, we need a dictionary whose key is site ip and value is a list of relevant user emails



## Import necessary packages and classes

In [1]:
# Retrieve user subscribing records from google sheets. Using Xinran version based on Ilija version.
from subscribers import subscribers
google = subscribers()

# Related to Elasticsearch queries
from elasticsearch import Elasticsearch, exceptions as es_exceptions, helpers
import sys
import datetime

# Regular Expression
import re

## Establish Elasticsearch connection

In [2]:
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

## List all alarms-yyyy.mm indices

In [3]:
indices = es.cat.indices(index="alarms-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
print(indices)

['alarms-2016.08']


## Find indices to be used

In [4]:
cday = datetime.datetime.utcnow()
pday = cday - datetime.timedelta(days=1)
ind1 = 'alarms-%d.%02d' % (cday.year, cday.month)
ind2 = 'alarms-%d.%02d' % (pday.year, pday.month)

print('checking for indices:', ind1, ind2)

ind=[]
if ind1 in indices:
    ind.append(ind1)
if ind2 != ind1 and ind2 in indices and cday.hour<3:   # not necessarily 3, just indicate it is the beginning period of new day
    ind.append(ind2)

if len(ind)==0:
    print('no current indices found. Aborting.')
    sys.exit(1)
else:
    print('will use indices:', ind)

checking for indices: alarms-2016.08 alarms-2016.08
will use indices: ['alarms-2016.08']


## Queries to find all the alarms of type Packet Loss for the past hour and past past hour

In [5]:
query_new = {
    "size": 1000,
    "query": {
        "bool": {
            "must": [
                {"term": { "_type": "packetloss" }}
            ],
            "filter": {
                "range": {
                    "alarmTime": {
                        "gt": "now-1h"
                    }
                }
            }
        }
    }
}

query_old = {
    "size": 1000,
    "query": {
        "bool": {
            "must": [
                {"term": { "_type": "packetloss" }}
            ],
            "filter": {
                "range": {
                    "alarmTime": {
                        "gt": "now-2h",
                        "lt": "now-1h"
                    }
                }
            }
        }
    }
}

print(query_new)
print(query_old)

{'size': 1000, 'query': {'bool': {'must': [{'term': {'_type': 'packetloss'}}], 'filter': {'range': {'alarmTime': {'gt': 'now-1h'}}}}}}
{'size': 1000, 'query': {'bool': {'must': [{'term': {'_type': 'packetloss'}}], 'filter': {'range': {'alarmTime': {'gt': 'now-2h', 'lt': 'now-1h'}}}}}}


## Execute the query

In [6]:
result_new = es.search(index=ind, body=query_new, request_timeout=120)
print('Number of hits of new alarms:', result_new['hits']['total'] )

result_old = es.search(index=ind, body=query_old, request_timeout=120)
print('Number of hits of old alarms:', result_old['hits']['total'] )

hits_new = result_new['hits']['hits']
hits_old = result_old['hits']['hits']

Number of hits of new alarms: 243
Number of hits of old alarms: 302


## Generate the two dictionaries for sites, one is from ip to name, one is from name to ip

In [7]:
site_ip_name = {}

for hit in hits_new:
    info = hit['_source']
    site_ip_name[info['src']] = info['srcSite']
    site_ip_name[info['dest']] = info['destSite']

for hit in hits_old:
    info = hit['_source']
    site_ip_name[info['src']] = info['srcSite']
    site_ip_name[info['dest']] = info['destSite']

site_ip_name

{'109.105.124.86': 'NDGF-T1',
 '109.105.125.232': 'FI_HIP_T2',
 '117.103.105.191': 'Taiwan-LCG2',
 '128.142.223.247': 'CERN-PROD',
 '128.227.221.44': 'UFlorida-HPC',
 '129.107.255.29': 'UTA_SWT2',
 '129.15.40.231': 'OU_OCHEP_SWT2',
 '129.215.213.70': 'UKI-SCOTGRID-ECDF',
 '129.93.239.148': 'Nebraska',
 '130.246.176.109': 'RAL-LCG2',
 '130.246.47.129': 'UKI-SOUTHGRID-RALPP',
 '131.111.66.196': 'UKI-SOUTHGRID-CAM-HEP',
 '131.154.254.12': 'INFN-T1',
 '131.169.98.30': 'DESY-HH',
 '131.225.205.12': 'UnknownSite',
 '132.195.125.213': 'wuppertalprod',
 '132.206.245.252': 'CA-MCGILL-CLUMEQ-T2',
 '132.230.202.235': 'UNI-FREIBURG',
 '134.158.103.10': 'IN2P3-LAPP',
 '134.158.123.183': 'IN2P3-LPC',
 '134.158.132.200': 'GRIF',
 '134.158.159.85': 'GRIF',
 '134.158.20.192': 'IN2P3-CPPM',
 '134.158.73.243': 'GRIF',
 '134.61.24.193': 'UnknownSite',
 '134.75.125.241': 'UnknownSite',
 '134.79.118.72': 'WT2',
 '137.222.74.15': 'UKI-SOUTHGRID-BRIS-HEP',
 '138.253.60.82': 'UKI-NORTHGRID-LIV-HEP',
 '141.108.

In [8]:
site_name_ip = {}

for ip in site_ip_name:
    name = site_ip_name[ip]
    if name in site_name_ip:
        site_name_ip[name].append(ip)
    else:
        site_name_ip[name] = [ip]

site_name_ip

{'AGLT2': ['192.41.236.31', '192.41.230.59'],
 'Australia-ATLAS': ['192.231.127.41'],
 'BEIJING-LCG2': ['202.122.32.170'],
 'BNL-ATLAS': ['192.12.15.26'],
 'BUDAPEST': ['148.6.8.251'],
 'BU_ATLAS_Tier2': ['192.5.207.251'],
 'CA-MCGILL-CLUMEQ-T2': ['132.206.245.252'],
 'CA-SCINET-T2': ['142.150.19.61'],
 'CA-VICTORIA-WESTGRID-T2': ['206.12.154.60'],
 'CERN-PROD': ['128.142.223.247'],
 'CSCS-LCG2': ['148.187.64.25'],
 'CYFRONET-LCG2': ['212.191.227.174'],
 'DESY-HH': ['131.169.98.30'],
 'DESY-ZN': ['141.34.200.28'],
 'EELA-UTFSM': ['146.83.90.7'],
 'FI_HIP_T2': ['109.105.125.232'],
 'FMPhI-UNIBA': ['158.195.14.26'],
 'FZK-LCG2': ['192.108.47.12'],
 'GLOW': ['144.92.180.75'],
 'GRIF': ['134.158.159.85',
  '134.158.132.200',
  '192.54.207.250',
  '134.158.73.243'],
 'IEPSAS-Kosice': ['147.213.204.112', '147.213.204.117'],
 'IFCA-LCG2': ['193.146.75.138'],
 'IFIC-LCG2': ['147.156.116.40'],
 'IN2P3-CC': ['193.48.99.76'],
 'IN2P3-CPPM': ['134.158.20.192'],
 'IN2P3-LAPP': ['134.158.103.10'],
 

## Calculate TN_old, the total number of alarmed links involving a specific site ip (either as source site or as destination site) for the OLD time period

In [9]:
TN_old = {}

def TN_old_add_one(ip):
    if ip in TN_old:
        TN_old[ip] += 1
    else:
        TN_old[ip] = 1

for alarm in hits_old:
    TN_old_add_one(alarm['_source']['src'])
    TN_old_add_one(alarm['_source']['dest'])

TN_old

{'109.105.124.86': 1,
 '109.105.125.232': 3,
 '117.103.105.191': 4,
 '128.142.223.247': 4,
 '128.227.221.44': 3,
 '129.107.255.29': 4,
 '129.15.40.231': 3,
 '129.215.213.70': 5,
 '129.93.239.148': 3,
 '130.246.176.109': 154,
 '130.246.47.129': 5,
 '131.111.66.196': 6,
 '131.154.254.12': 5,
 '131.169.98.30': 5,
 '131.225.205.12': 3,
 '132.195.125.213': 1,
 '132.206.245.252': 4,
 '132.230.202.235': 2,
 '134.158.103.10': 2,
 '134.158.123.183': 5,
 '134.158.132.200': 3,
 '134.158.159.85': 3,
 '134.158.20.192': 4,
 '134.158.73.243': 4,
 '134.61.24.193': 2,
 '134.75.125.241': 2,
 '134.79.118.72': 2,
 '137.222.74.15': 2,
 '138.253.60.82': 4,
 '141.108.35.18': 19,
 '141.34.200.28': 4,
 '142.150.19.61': 3,
 '143.167.3.116': 1,
 '143.215.129.69': 1,
 '144.16.111.26': 3,
 '144.92.180.75': 2,
 '145.100.17.8': 5,
 '146.83.90.7': 4,
 '147.156.116.40': 1,
 '147.213.204.112': 1,
 '147.213.204.117': 1,
 '147.231.25.192': 4,
 '148.187.64.25': 5,
 '148.6.8.251': 2,
 '149.165.225.223': 3,
 '150.244.247.2'

## Calculate TN_new, the total number of alarmed links involving a specific site ip (either as source site or as destination site) for the NEW time period

In [10]:
TN_new = {}

def TN_new_add_one(ip):
    if ip in TN_new:
        TN_new[ip] += 1
    else:
        TN_new[ip] = 1

for alarm in hits_new:
    TN_new_add_one(alarm['_source']['src'])
    TN_new_add_one(alarm['_source']['dest'])

TN_new

{'109.105.125.232': 3,
 '117.103.105.191': 4,
 '128.142.223.247': 3,
 '128.227.221.44': 2,
 '129.107.255.29': 3,
 '129.15.40.231': 3,
 '129.215.213.70': 5,
 '129.93.239.148': 2,
 '130.246.176.109': 88,
 '130.246.47.129': 4,
 '131.111.66.196': 5,
 '131.154.254.12': 4,
 '131.169.98.30': 4,
 '131.225.205.12': 2,
 '132.195.125.213': 2,
 '132.206.245.252': 3,
 '132.230.202.235': 2,
 '134.158.103.10': 2,
 '134.158.123.183': 4,
 '134.158.132.200': 4,
 '134.158.159.85': 3,
 '134.158.20.192': 4,
 '134.158.73.243': 2,
 '134.61.24.193': 2,
 '134.75.125.241': 1,
 '134.79.118.72': 1,
 '137.222.74.15': 2,
 '138.253.60.82': 6,
 '141.108.35.18': 18,
 '141.34.200.28': 3,
 '142.150.19.61': 2,
 '143.167.3.116': 1,
 '143.215.129.69': 1,
 '144.16.111.26': 1,
 '144.92.180.75': 1,
 '145.100.17.8': 4,
 '146.83.90.7': 3,
 '147.213.204.112': 5,
 '147.231.25.192': 4,
 '148.187.64.25': 4,
 '148.6.8.251': 1,
 '149.165.225.223': 2,
 '150.244.247.2': 1,
 '152.84.101.141': 3,
 '157.82.112.68': 3,
 '158.195.14.26': 3,

## Calculate TN_delta, which is equal to ( TN_new - TN_old )

In [11]:
TN_delta = {}

for ip in TN_old:
    if ip in TN_new:
        TN_delta[ip] = TN_new[ip] - TN_old[ip]
    else:
        TN_delta[ip] = -TN_old[ip]

for ip in TN_new:
    if ip not in TN_old:
        TN_delta[ip] = TN_new[ip]

TN_delta

{'109.105.124.86': -1,
 '109.105.125.232': 0,
 '117.103.105.191': 0,
 '128.142.223.247': -1,
 '128.227.221.44': -1,
 '129.107.255.29': -1,
 '129.15.40.231': 0,
 '129.215.213.70': 0,
 '129.93.239.148': -1,
 '130.246.176.109': -66,
 '130.246.47.129': -1,
 '131.111.66.196': -1,
 '131.154.254.12': -1,
 '131.169.98.30': -1,
 '131.225.205.12': -1,
 '132.195.125.213': 1,
 '132.206.245.252': -1,
 '132.230.202.235': 0,
 '134.158.103.10': 0,
 '134.158.123.183': -1,
 '134.158.132.200': 1,
 '134.158.159.85': 0,
 '134.158.20.192': 0,
 '134.158.73.243': -2,
 '134.61.24.193': 0,
 '134.75.125.241': -1,
 '134.79.118.72': -1,
 '137.222.74.15': 0,
 '138.253.60.82': 2,
 '141.108.35.18': -1,
 '141.34.200.28': -1,
 '142.150.19.61': -1,
 '143.167.3.116': 0,
 '143.215.129.69': 0,
 '144.16.111.26': -2,
 '144.92.180.75': -1,
 '145.100.17.8': -1,
 '146.83.90.7': -1,
 '147.156.116.40': -1,
 '147.213.204.112': 4,
 '147.213.204.117': -1,
 '147.231.25.192': 0,
 '148.187.64.25': -1,
 '148.6.8.251': -1,
 '149.165.225.

## Look at the distribution of TN_delta, so that we can tune the parameter N

In [12]:
for N in range(10):
    count_worse = 0
    count_better = 0
    count_stable = 0
    for ip in TN_delta:
        if TN_delta[ip] >= N:
            count_worse += 1
        elif TN_delta[ip] <= -N:
            count_better += 1
        else:
            count_stable += 1
    print('N=%d     count_worse=%d     count_better=%d     count_stable=%d' % (N, count_worse, count_better, count_stable))

N=0     count_worse=42     count_better=56     count_stable=0
N=1     count_worse=15     count_better=56     count_stable=27
N=2     count_worse=4     count_better=12     count_stable=82
N=3     count_worse=2     count_better=6     count_stable=90
N=4     count_worse=2     count_better=4     count_stable=92
N=5     count_worse=0     count_better=2     count_stable=96
N=6     count_worse=0     count_better=1     count_stable=97
N=7     count_worse=0     count_better=1     count_stable=97
N=8     count_worse=0     count_better=1     count_stable=97
N=9     count_worse=0     count_better=1     count_stable=97


## Let's use N=3 for now, and we will tune later

In [13]:
N = 3

ip_list_worse = []
ip_list_better = []

for ip in TN_delta:
    if TN_delta[ip] >= N:
        ip_list_worse.append(ip)
    elif TN_delta[ip] <= -N:
        ip_list_better.append(ip)

print('--- The ip of the site(s) which got worse:')
print(ip_list_worse)
print('--- The ip of the site(s) which got better:')
print(ip_list_better)

--- The ip of the site(s) which got worse:
['147.213.204.112', '193.206.93.45']
--- The ip of the site(s) which got better:
['130.246.176.109', '194.85.69.75', '193.48.83.165', '206.12.24.251', '192.54.207.250', '195.194.105.178']


## Generate the dictionary: key = site name, value = a list of relevant user emails

In [14]:
user_interest_site_name = {}

def reg_user_interest_site_name(sitename, email):
    if sitename in user_interest_site_name:
        user_interest_site_name[sitename].append(email)
    else:
        user_interest_site_name[sitename] = [email]

taskName = 'Packet loss increase for link(s) where your site is a source or destination'

subscribe_records = google.getSubscribers_withSiteName(taskName)

# Handle blank answer, one site, several sites separated by comma, wildcard such as prefix* etc.
for record in subscribe_records:
    email = record[1]
    sitenames = record[3].strip()
    if len(sitenames) == 0:
        sitenames = '.'  # Handle blank answer, so match all site names
    sitenames = [x.strip().replace('*', '.') for x in sitenames.split(',')]  # Handle several site names, and wildcard
    for sn in sitenames:
        p = re.compile(sn, re.IGNORECASE)
        for sitename in site_name_ip:
            if p.match(sitename):
                reg_user_interest_site_name(sitename, email)

user_interest_site_name

{'AGLT2': ['ivukotic@cern.ch', 'elancon@bnl.gov', 'marian.babik@cern.ch'],
 'Australia-ATLAS': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'BEIJING-LCG2': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'BNL-ATLAS': ['ivukotic@cern.ch', 'elancon@bnl.gov', 'marian.babik@cern.ch'],
 'BUDAPEST': ['ivukotic@cern.ch', 'elancon@bnl.gov', 'marian.babik@cern.ch'],
 'BU_ATLAS_Tier2': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'CA-MCGILL-CLUMEQ-T2': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'CA-SCINET-T2': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'CA-VICTORIA-WESTGRID-T2': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'CERN-PROD': ['ivukotic@cern.ch',
  'xinran@uchicago.edu',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 'CSCS-LCG2': ['ivukotic@cern.ch', 'elancon@bnl.gov', 'marian.babik@cern.ch'],
 'CYFRONET-LCG2': ['ivukotic@cern.ch',
  '

## Generate the dictionary: key = site ip, value = a list of relevant user emails

In [15]:
user_interest_site_ip = {}

def reg_user_interest_site_ip(siteip, email):
    if siteip in user_interest_site_ip:
        user_interest_site_ip[siteip].append(email)
    else:
        user_interest_site_ip[siteip] = [email]

for sitename in user_interest_site_name:
    for siteip in site_name_ip[sitename]:
        for email in user_interest_site_name[sitename]:
            reg_user_interest_site_ip(siteip, email)

user_interest_site_ip

{'109.105.124.86': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '109.105.125.232': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '117.103.105.191': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '128.142.223.247': ['ivukotic@cern.ch',
  'xinran@uchicago.edu',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '128.227.221.44': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '129.107.255.29': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '129.15.40.231': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '129.215.213.70': ['ivukotic@cern.ch',
  'xinran@uchicago.edu',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '129.93.239.148': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '130.246.176.109': ['ivukotic@cern.ch',
  'elancon@bnl.gov',
  'marian.babik@cern.ch'],
 '130.246.47.129': ['ivukotic@cern.ch',
  'xinran@uchicago.edu',
  

## The variable user_alert_all holds all the needed info to send an email to a specific user

In [16]:
user_alert_all = {}

for user in google.getAllUserBasicInfo():
    user_info = {}
    user_info['email'] = user[0]
    user_info['fullname'] = user[1]
    user_info['link'] = user[2]
    user_info['alerts'] = []
    user_alert_all[user[0]] = user_info    # email should be unique globally, so it is used as key

user_alert_all

{'LINCOLNB@UCHICAGO.EDU': {'alerts': [],
  'email': 'LINCOLNB@UCHICAGO.EDU',
  'fullname': 'Lincoln Bryant',
  'link': 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnufEkC3ZkGEJM7YrjIK5M9zZCa8Fg0C58snIuKzECJRbScqS5po96bBVKA'},
 'elancon@bnl.gov': {'alerts': [],
  'email': 'elancon@bnl.gov',
  'fullname': 'Eric Lancon',
  'link': 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuf_umrjRScBiYvJ_aUfteZA2eiV-yUK9DMa3q7VefVJtC5bEQnNSPOg4A'},
 'ilijav@gmail.com': {'alerts': [],
  'email': 'ilijav@gmail.com',
  'fullname': 'Ilija Vukotic',
  'link': 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuezuteui57-PrydNWrUuZf5fmChNqtjEeDab6h5V6lik_-x790uKsPu5Q'},
 'ivukotic@cern.ch': {'alerts': [],
  'email': 'ivukotic@cern.ch',
  'fullname': 'Testing deleting CERN',
  'link': 'https://docs.google.com/for

## Generate info for sending alert emails (for the sites getting worse)

In [17]:
for ip in ip_list_worse:
    text = "The site %s (%s)'s links got worse, total number from %d to %d links.\n" % (site_ip_name[ip], ip, TN_old.get(ip,0), TN_new.get(ip,0))
    text += "These are all the bad links for the past hour:\n"
    for alarm in hits_new:
        src_ip = alarm['_source']['src']
        dest_ip = alarm['_source']['dest']
        if src_ip == ip:
            text += '    This site  --->  %s (%s) \n' % (site_ip_name[dest_ip], dest_ip)
    for alarm in hits_new:
        src_ip = alarm['_source']['src']
        dest_ip = alarm['_source']['dest']
        if dest_ip == ip:
            text += '    %s (%s)  --->  This site \n' % (site_ip_name[src_ip], src_ip)
    print(text)
    for email in user_interest_site_ip[ip]:
        user_alert_all[email]['alerts'].append(text)

# user_alert_all

The site IEPSAS-Kosice (147.213.204.112)'s links got worse, total number from 1 to 5 links.
These are all the bad links for the past hour:
    RO-02-NIPNE (81.180.86.38)  --->  This site 
    pic (193.109.172.188)  --->  This site 
    IFCA-LCG2 (193.146.75.138)  --->  This site 
    UKI-SOUTHGRID-OX-HEP (163.1.5.210)  --->  This site 
    Australia-ATLAS (192.231.127.41)  --->  This site 

The site UnknownSite (193.206.93.45)'s links got worse, total number from 5 to 9 links.
These are all the bad links for the past hour:
    This site  --->  UKI-NORTHGRID-MAN-HEP (195.194.105.178) 
    This site  --->  IN2P3-CC (193.48.99.76) 
    This site  --->  RAL-LCG2 (130.246.176.109) 
    This site  --->  UnknownSite (161.116.81.235) 
    This site  --->  UnknownSite (192.101.107.152) 
    This site  --->  NCG-INGRID-PT (193.136.75.146) 
    BEIJING-LCG2 (202.122.32.170)  --->  This site 
    UKI-NORTHGRID-LIV-HEP (138.253.60.82)  --->  This site 
    RAL-LCG2 (130.246.176.109)  --->  This sit

## Generate info for sending alert emails (for the sites getting better)

In [18]:
for ip in ip_list_better:
    text = "The site %s (%s)'s links got improved, total number from %d to %d links.\n" % (site_ip_name[ip], ip, TN_old.get(ip,0), TN_new.get(ip,0))
    text += "These are all the bad links for the past hour:\n"
    for alarm in hits_new:
        src_ip = alarm['_source']['src']
        dest_ip = alarm['_source']['dest']
        if src_ip == ip:
            text += '    This site  --->  %s (%s) \n' % (site_ip_name[dest_ip], dest_ip)
    for alarm in hits_new:
        src_ip = alarm['_source']['src']
        dest_ip = alarm['_source']['dest']
        if dest_ip == ip:
            text += '    %s (%s)  --->  This site \n' % (site_ip_name[src_ip], src_ip)
    print(text)
    for email in user_interest_site_ip[ip]:
        user_alert_all[email]['alerts'].append(text)

# user_alert_all

The site RAL-LCG2 (130.246.176.109)'s links got improved, total number from 154 to 88 links.
These are all the bad links for the past hour:
    This site  --->  Taiwan-LCG2 (117.103.105.191) 
    This site  --->  GRIF (134.158.159.85) 
    This site  --->  UKI-SCOTGRID-ECDF (129.215.213.70) 
    This site  --->  GRIF (134.158.132.200) 
    This site  --->  Australia-ATLAS (192.231.127.41) 
    This site  --->  UKI-NORTHGRID-LANCS-HEP (194.80.35.169) 
    This site  --->  UnknownSite (165.91.55.4) 
    This site  --->  FZK-LCG2 (192.108.47.12) 
    This site  --->  UKI-SCOTGRID-DURHAM (193.60.193.3) 
    This site  --->  UnknownSite (90.147.66.50) 
    This site  --->  FI_HIP_T2 (109.105.125.232) 
    This site  --->  Vanderbilt (192.111.108.112) 
    This site  --->  SPRACE (200.136.80.20) 
    This site  --->  INFN-ROMA1 (141.108.35.18) 
    This site  --->  UKI-LT2-QMUL (194.36.11.38) 
    This site  --->  MWT2 (149.165.225.223) 
    This site  --->  UnknownSite (193.206.93.45) 
    

## Dummy sendMail function for development purpose

In [19]:
def sendMailDummy(subject, to, body):
    if len(body['alerts']) == 0:
        print('======== Do not send alert email to %s as there is no alert for this user ========' % to)
    else:
        print('========= Send the following email to a user =========')
        print('------ Email subject ------')
        subject = 'Alert email customized for %s' % body['fullname']
        print(subject)
        print('------ Email to -----------')
        print(to)
        print('------ Email body ---------')
        text = 'Hi %s,\n\n' % body['fullname']
        text += '    The following are all the alerts about packet loss that you are interested in:\n\n\n'
        for alert in body['alerts']:
            text += alert
            text += '\n\n'
        text += 'Thank you for using this system. If you want to update your settings or unsubscribe, please use this link: %s' % body['link']
        text += '\n\nBest,\nThe team\n\n\n'
        print(text)
        print('======================================================')
    print()
    print()

## Send out alert email customized for each user

In [20]:
for email in user_alert_all:
    sendMailDummy('auto_subject', email, user_alert_all[email])

========= Send the following email to a user =========
------ Email subject ------
Alert email customized for Xinran Wang
------ Email to -----------
xinran@uchicago.edu
------ Email body ---------
Hi Xinran Wang,

    The following are all the alerts about packet loss that you are interested in:


The site SFU-LCG2 (206.12.24.251)'s links got improved, total number from 3 to 0 links.
These are all the bad links for the past hour:


The site UKI-NORTHGRID-MAN-HEP (195.194.105.178)'s links got improved, total number from 36 to 31 links.
These are all the bad links for the past hour:
    This site  --->  UnknownSite (161.116.81.235) 
    This site  --->  RAL-LCG2 (130.246.176.109) 
    INFN-T1 (131.154.254.12)  --->  This site 
    CA-MCGILL-CLUMEQ-T2 (132.206.245.252)  --->  This site 
    AGLT2 (192.41.236.31)  --->  This site 
    UnknownSite (193.205.76.76)  --->  This site 
    praguelcg2 (147.231.25.192)  --->  This site 
    CYFRONET-LCG2 (212.191.227.174)  --->  This site 
    RO